### Retrieve FEWSNET Data:

In [17]:
import requests
url_csv = "https://fdw.fews.net/api/ipcphase.csv?start_date=2000-01-01&end_date=2024-12-31&classification_scale=IPC31"

url_ethiopia = 'https://fdw.fews.net/api/ipcphase/?format=csv&classification_scale=IPC31&country_code=ET'

url_csv_metadata = "https://fdw.fews.net/api/ipcphase/metadata/?start_date=2000-01-01&end_date=2024-12-31&classification_scale=IPC31"

url_geometry = 'https://fdw.fews.net/api/feature/?format=geojson&fields=with_attributes&unit_set_version=3528'

params = {
    "start_date": "2000-01-01",
    "end_date": "2024-12-31",
    "classification_scale": "IPC31"
}

response = requests.get(url_csv, params=params)
with open("ipc_data.csv", "wb") as file:
    file.write(response.content)


print("Data downloaded and saved as ipc_data.csv")

Data downloaded and saved as ipc_data.csv


In [19]:
import pandas as pd

#ipc_data_ethiopia = pd.read_csv('ipc_ethiopia.csv')
ipc_data = pd.read_csv('ipc_data.csv')

In [20]:
#print(list(ipc_data_ethiopia))
print(list(ipc_data))

['source_organization', 'source_document', 'country', 'country_code', 'geographic_group', 'fewsnet_region', 'geographic_unit_full_name', 'geographic_unit_name', 'unit_type', 'fnid', 'classification_scale', 'scenario_name', 'preference_rating', 'is_allowing_for_assistance', 'projection_start', 'projection_end', 'status', 'value', 'pct_phase3', 'pct_phase4', 'pct_phase5', 'description', 'id', 'datacollectionperiod', 'datacollection', 'scenario', 'geographic_unit', 'datasourceorganization', 'datasourcedocument', 'dataseries', 'dataseries_name', 'specialization_type', 'dataseries_specialization_type', 'data_usage_policy', 'created', 'modified', 'status_changed', 'collection_status', 'collection_status_changed', 'collection_schedule', 'reporting_date']


In [93]:
pd.set_option('display.max_columns', None)  # Show all columns

ipc_data_ethiopia

,source_organization,source_document,country,country_code,geographic_group,fewsnet_region,geographic_unit_full_name,geographic_unit_name,unit_type,fnid,classification_scale,scenario_name,preference_rating,is_allowing_for_assistance,projection_start,projection_end,status,value,pct_phase3,pct_phase4,pct_phase5,description,id,datacollectionperiod,datacollection,scenario,geographic_unit,datasourceorganization,datasourcedocument,dataseries,dataseries_name,specialization_type,dataseries_specialization_type,data_usage_policy,created,modified,status_changed,collection_status,collection_status_changed,collection_schedule,reporting_date
0,FEWS NET,"Food Security Outlook, Ethiopia",Ethiopia,ET,Eastern Africa,East Africa,"Abaala, Kilbati, Afar, Ethiopia",Abaala,fsc_admin,ET2023C1020203,IPC 3.1,Current Situation,90,False,2023-10-01,2023-10-31,Collected,4.0,NaN,NaN,NaN,Emergency,33853165,323962,338763,CS,224045,1,6537,6978571,"Abaala, Kilbati, Afar, Ethiopia (From 10/01/20...",/IPCPhase/,/IPCClassification/,Public,2023-12-01T16:12:04,2023-12-01T18:11:32,2023-12-01T18:11:32,Published,2023-12-01T18:48:20,Ad Hoc,2023-10-01
1,FEWS NET,"Food Security Outlook, Ethiopia",Ethiopia,ET,Eastern Africa,East Africa,"Abaala, Kilbati, Afar, Ethiopia",Abaala,fsc_admin,ET2023C1020203,IPC 3.1,Near Term Projection,85,False,2023-10-01,2024-01-31,Collected,4.0,NaN,NaN,NaN,Emergency,33854306,323963,338763,ML1,224045,1,6537,6979712,"Abaala, Kilbati, Afar, Ethiopia (From 10/01/20...",/IPCPhase/,/IPCClassification/,Public,2023-12-01T16:12:24,2023-12-01T18:12:16,2023-12-01T18:12:16,Published,2023-12-01T18:48:20,Ad Hoc,2023-10-01
2,FEWS NET,"Food Security Outlook, Ethiopia",Ethiopia,ET,Eastern Africa,East Africa,"Abaala, Kilbati, Afar, Ethiopia",Abaala,fsc_admin,ET2023C1020203,IPC 3.1,Current Situation,90,False,2024-02-01,2024-02-29,Collected,4.0,NaN,NaN,NaN,Emergency,34855628,330458,344610,CS,224045,1,6537,6978571,"Abaala, Kilbati, Afar, Ethiopia (From 10/01/20...",/IPCPhase/,/IPCClassification/,Public,2024-03-29T14:38:54,2024-03-29T14:38:54,2024-03-29T14:39:52,Published,2024-03-29T15:14:29,Ad Hoc,2024-02-01
3,FEWS NET,"Food Security Outlook, Ethiopia",Ethiopia,ET,Eastern Africa,East Africa,"Abaala, Kilbati, Afar, Ethiopia",Abaala,fsc_admin,ET2023C1020203,IPC 3.1,Medium Term Projection,80,False,2024-02-01,2024-05-31,Collected,4.0,NaN,NaN,NaN,Emergency,33855447,323964,338763,ML2,224045,1,6537,6980853,"Abaala, Kilbati, Afar, Ethiopia (From 10/01/20...",/IPCPhase/,/IPCClassification/,Public,2023-12-01T16:12:44,2023-12-01T18:13:00,2023-12-01T18:13:00,Published,2023-12-01T18:48:20,Ad Hoc,2023-10-01
4,FEWS NET,"Food Security Outlook, Ethiopia",Ethiopia,ET,Eastern Africa,East Africa,"Abaala, Kilbati, Afar, Ethiopia",Abaala,fsc_admin,ET2023C1020203,IPC 3.1,Near Term Projection,85,False,2024-02-01,2024-05-31,Collected,4.0,NaN,NaN,NaN,Emergency,34856769,330459,344610,ML1,224045,1,6537,6979712,"Abaala, Kilbati, Afar, Ethiopia (From 10/01/20...",/IPCPhase/,/IPCClassification/,Public,2024-03-29T14:39:14,2024-03-29T14:39:14,2024-03-29T14:39:52,Published,2024-03-29T15:14:29,Ad Hoc,2024-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73947,FEWS NET,"Key Message Update, Ethiopia",Ethiopia,ET,Eastern Africa,East Africa,"Ziway Dugda, Arsi, Oromia, Ethiopia",Ziway Dugda,fsc_admin,ET2023C1040806,IPC 3.1,Medium Term Projection,80,False,2024-10-01,2025-01-31,Collected,1.0,NaN,NaN,NaN,Minimal,36597704,340011,351753,ML2,224448,1,6540,7002248,"Ziway Dugda, Arsi, Oromia, Ethiopia (From 10/0...",/IPCPhase/,/IPCClassification/,Public,2024-08-20T21:14:36,2024-08-20T21:14:36,2024-08-20T21:14:48,Published,2024-08-20T21:17:07,Ad Hoc,2024-07-01
73948,FEWS NET,"Key Message Update, Ethiopia",Ethiopia,ET,Eastern Africa,East Africa,"Ziway Dugda, Arsi, Oromia, Ethiopia",Ziway Dugda,fsc_admin,ET2023C1040806,IPC 3.1,Medium Term Projection,80,False,2024-10-01,2025-01-31,Collected,1.0

### Investigate the temporal distribution (how much of the data was collected and when) 

this does not inherently mean there is detrimental conditions, but a measure of the robustness of data collection at different time periods. 

### Investigate the distribution of `unit_type`  

In [94]:
units_list = list(pd.unique(ipc_data['unit_type']))
units_list

['fsc_admin', 'fsc_admin_lhz', 'idp_camp', 'national_park', 'fsc_rm_admin']

### Match the appropriate Admin unit to the shapefile boundary

- load the shapefiles:

In [14]:
import requests
import geopandas as gpd
from io import BytesIO

# URL of the GeoJSON file
url = 'https://fdw.fews.net/api/feature/?format=geojson&fields=with_attributes&country_code=ET&unit_type=idp_camp&unit_type=livelihood_zone&unit_type=national_park&unit_type=fsc_admin&unit_type=fsc_admin_lhz&unit_type=fsc_lhz&unit_type=fsc_rm_admin'
# Fetch the GeoJSON file
response = requests.get(url)

if response.status_code == 200:
    # Load the GeoJSON into a GeoPandas DataFrame
    gdf__fscunits = gpd.read_file(BytesIO(response.content))
    print(gdf__fscunits.head())
else:
    print(f"Failed to fetch data: {response.status_code}")

unique_countries = sorted(gdf__fscunits['country_code'].unique().tolist())
unique_countries

       id            fnid            name unit_type_code parent_id  \
0  223937  ET2023C1010101  Tahtay Adiyabo      fsc_admin      None   
1  223955  ET2023C1010204            Adwa      fsc_admin      None   
2  223956  ET2023C1010205  Laelay Maychew      fsc_admin      None   
3  223959  ET2023C1010208     Kola Temben      fsc_admin      None   
4  223965  ET2023C1010214            Rama      fsc_admin      None   

   start_date end_date country_code fewsnet_region geographic_group  ...  \
0  2023-10-01     None           ET    East Africa   Eastern Africa  ...   
1  2023-10-01     None           ET    East Africa   Eastern Africa  ...   
2  2023-10-01     None           ET    East Africa   Eastern Africa  ...   
3  2023-10-01     None           ET    East Africa   Eastern Africa  ...   
4  2023-10-01     None           ET    East Africa   Eastern Africa  ...   

  layer_id         layer_name                 geographicunitsetversion_name  \
0     4698  ET_Admin3C_2023_3  Ethiopia FEW

['ET']

In [5]:
import geopandas as gpd

# Provide the path to your shapefile
admin2_path = '/Users/gbenz/Documents/FEWSnet/FEWSNET_World_Admin/FEWSNET_Admin2.shp'
admin1_path = '/Users/gbenz/Documents/FEWSnet/FEWSNET_World_Admin/FEWSNET_Admin1.shp'

# Load the shapefile into a GeoDataFrame
gdf__admin2 = gpd.read_file(admin2_path)
gdf__admin1 = gpd.read_file(admin1_path)
# Display the first few rows to verify
print(gdf__admin2.head())
print(gdf__admin1.head())


DriverError: 'ipc_geometry.geojson' not recognized as a supported file format.

In [21]:
subset_df_admin2 = gdf__admin2[gdf__admin2['ADMIN0'] == 'Ethiopia']
subset_df_admin1 = gdf__admin1[gdf__admin1['ADMIN0'] == 'Ethiopia']

subset_df = ipc_data[ipc_data['country'] == 'Ethiopia']


In [23]:
#print(subset_df_admin2.head())
print(subset_df.head())

#compare FNID
fnid_admin1 = subset_df_admin1['FNID'].drop_duplicates().to_list()
fnid_admin2 = subset_df_admin2['FNID'].drop_duplicates().to_list()
fnid_fscunits = gdf__fscunits['fnid'].drop_duplicates().to_list()

fnid_fewsnet = subset_df['fnid'].drop_duplicates().to_list()

# Compare names
fnid_admin1_name = sorted(subset_df_admin1['ADMIN1'].drop_duplicates().to_list())
fnid_admin2_name = sorted(subset_df_admin2['ADMIN2'].drop_duplicates().to_list())
fnid_fewsnet_name = sorted(subset_df['geographic_unit_name'].drop_duplicates().to_list())

#print fnid lists
print('printing FNID for Admin1:')
print(fnid_admin1)
print()
print('printing FNID for Admin2:')
print(fnid_admin2)
print()
print('printing FNID for FSC data:')
print(fnid_fscunits)
print()
print('printing FNID for FEWSNET Data:')
print(fnid_fewsnet)
print()
# print admin names list
print(fnid_admin1_name)
print(fnid_admin2_name)
print(fnid_fewsnet_name)

  source_organization                source_document   country country_code  \
0            FEWS NET  Food Security Alert, Ethiopia  Ethiopia           ET   
1            FEWS NET  Food Security Alert, Ethiopia  Ethiopia           ET   
2            FEWS NET  Food Security Alert, Ethiopia  Ethiopia           ET   
3            FEWS NET  Food Security Alert, Ethiopia  Ethiopia           ET   
4            FEWS NET  Food Security Alert, Ethiopia  Ethiopia           ET   

  geographic_group fewsnet_region  \
0   Eastern Africa    East Africa   
1   Eastern Africa    East Africa   
2   Eastern Africa    East Africa   
3   Eastern Africa    East Africa   
4   Eastern Africa    East Africa   

                     geographic_unit_full_name geographic_unit_name  \
0       Abaala, Kilbati /Zone2, Afar, Ethiopia               Abaala   
1       Abaala, Kilbati /Zone2, Afar, Ethiopia               Abaala   
2  Abaala town, Kilbati /Zone2, Afar, Ethiopia          Abaala town   
3  Abaala town, Ki

In [24]:
# Convert to sets for comparison
set_1 = set(fnid_fewsnet)
set_2 = set(fnid_fscunits)

# 1. Count matching values
matching_values = set_1 & set_2  # Intersection of sets
num_matching = len(matching_values)

# 2. List unmatched values from fnid_fewsnet_1
unmatched_values = set_1 - set_2  # Values in set_1 but not in set_2

# Print results
print(f"Number of matching values: {num_matching}")
print(f"Matching values: {matching_values}")
print(f"Unmatched values from fnid_fewsnet_1: {unmatched_values}")

Number of matching values: 2181
Matching values: {'ET2021C1031109', 'ET2021C1041703', 'ET2021C1030119', 'ET2021C1160011', 'ET2023C1041408', 'ET2023C1041703', 'ET2023C1040119', 'ET2021C1130108', 'ET2021C1050599', 'ET2021C1010112', 'ET2023C1010602', 'ET2023C1041605', 'ET2023C1050408', 'ET2021C1030214', 'ET2021C1040711', 'ET2021C1072008', 'ET2023C1180505', 'ET2023C1041017', 'ET2023C1040490', 'ET2023C1150207', 'ET2023C1020201', 'ET2021C1041701', 'ET2023C1192505', 'ET2023C1040799', 'ET2021C1160007', 'ET2021C1041212', 'ET2023C1170909', 'ET2021C1041596', 'ET2023C1040513', 'ET2023C1030801', 'ET2021C1030604', 'ET2023C1010304', 'ET2021C1010222', 'ET2021C1050304', 'ET2023C1030999', 'ET2021C1070608', 'ET2021C1050302', 'ET2023C1042310', 'ET2021C1050898', 'ET2021C1070204', 'ET2021C1040825', 'ET2021C1030709', 'ET2021C1160004', 'ET2023C1130105', 'ET2021C1140108', 'ET2023C1030305', 'ET2023C1040212', 'ET2023C1030699', 'ET2023C1181005', 'ET2021C1041515', 'ET2021C1160012', 'ET2021C1042018', 'ET2021C107021

### Simplify these admin files:
so only the FNID and Effective Year (EFF_YEAR) and geometry attributes are present in the datafames

- then we will join from the FEWSNET data to declare the extent to which the FEWSNET FNID can locate matches at either Admin 1 or Admin 2 levels. 

- From FEWSNET: compare admin fields with fnid and reporting_date fields
    - reporting_date is in YYYY-MM-DD Format. This will need to be corrected. 

In [40]:
subset_fewsnet = ipc_data[['fnid', 'reporting_date']].drop_duplicates()
subset_fewsnet['reporting_date'] = subset_fewsnet['reporting_date'].astype(str)
subset_fewsnet['year'] = subset_fewsnet['reporting_date'].str[:4]
# Retrieve ONLY admin strings ---
adminonly_fewsnet = subset_fewsnet[['fnid']].drop_duplicates()
# ----------------------------------------------------------------------
# ADMIN 1
# ----------------------------------------------------------------------
subset_admin1 = gdf__admin1[['FNID', 'EFF_YEAR']].drop_duplicates()
admin1_eff_years = sorted(subset_admin1['EFF_YEAR'].unique())
# Retrieve ONLY admin strings ---
adminonly_admin1 = subset_admin1[['FNID']].drop_duplicates()
# ----------------------------------------------------------------------
# Admin 2
# ----------------------------------------------------------------------
subset_admin2 = gdf__admin2[['FNID', 'EFF_YEAR']].drop_duplicates()
admin2_eff_years = sorted(subset_admin2['EFF_YEAR'].unique())
# Retrieve ONLY admin strings ---
adminonly_admin2 = subset_admin2[['FNID']].drop_duplicates()
# ----------------------------------------------------------------------

In [41]:
print(adminonly_fewsnet)
print()
print(adminonly_admin1)
print()
print(adminonly_admin2)

                  fnid
0       ET2021C1020203
2       ET2021C1020209
4       ET2021C1041904
6       ET2021C1130109
8       ET2021C1050699
...                ...
335015  BF2021C3500203
335017  BF2021C3530303
338789        VE2023C0
338821        LB2024C0
338833        TG2024C0

[7456 rows x 1 columns]

           FNID
0    AO2008A101
1    AO2008A102
2    AO2008A103
3    AO2008A104
4    AO2008A105
..          ...
664  AF2017A122
665  AF2017A118
666  AF2017A125
667  AF2017A104
668  AF2017A126

[668 rows x 1 columns]

              FNID
0     AO2008A21701
1     AO2008A21501
2     AO2008A20901
3     AO2008A21001
4     AO2008A20101
...            ...
6560  AF2017A20105
6561  AF2017A20907
6562  AF2017A21713
6563  AF2017A20102
6564  AF2017A21816

[6562 rows x 1 columns]


## See if matches exists between admin attributes:

In [43]:
# Assuming the lists `adminonly_fewsnet` and `adminonly_admin1` are defined

# Convert the lists to sets for comparison
set_fewsnet = set(adminonly_fewsnet)
set_admin2 = set(adminonly_admin2)

# Find the intersection (matches)
matches = set_fewsnet.intersection(set_admin2)

# Print the matches and their count
print("Matches between adminonly_fewsnet and adminonly_admin1:")
print(matches)
print(f"\nNumber of matches: {len(matches)}")


Matches between adminonly_fewsnet and adminonly_admin1:
set()

Number of matches: 0


## Shows ZERO matches
- we need to identify WHY no matches exist

#### What other field might provide a match?
- geographic_unit_name (subset_FEWSNET)
- ADMIN1 (admin1 dataframe)
- ADMIN2 (admin2 dataframe)

In [47]:
subset_fewsnet_name = ipc_data[['geographic_unit_name', 'reporting_date']].drop_duplicates()
subset_fewsnet_name['reporting_date'] = subset_fewsnet_name['reporting_date'].astype(str)
subset_fewsnet_name['year'] = subset_fewsnet_name['reporting_date'].str[:4]
# Retrieve ONLY admin strings ---
admin_name_only_fewsnet = subset_fewsnet_name[['geographic_unit_name']].drop_duplicates()
# ----------------------------------------------------------------------
# ADMIN 1
# ----------------------------------------------------------------------
subset_admin1_name = gdf__admin1[['ADMIN1PT', 'EFF_YEAR']].drop_duplicates()
admin1_eff_years = sorted(subset_admin1_name['EFF_YEAR'].unique())
# Retrieve ONLY admin strings ---
admin_name_only_admin1 = subset_admin1_name[['ADMIN1PT']].drop_duplicates()
# ----------------------------------------------------------------------
# Admin 2
# ----------------------------------------------------------------------
subset_admin2_name = gdf__admin2[['ADMIN2PT', 'EFF_YEAR']].drop_duplicates()
admin2_eff_years = sorted(subset_admin2_name['EFF_YEAR'].unique())
# Retrieve ONLY admin strings ---
admin_name_only_admin2 = subset_admin2_name[['ADMIN2PT']].drop_duplicates()
# ----------------------------------------------------------------------

### Again, check if matches exist:

In [48]:
# Assuming the lists `adminonly_fewsnet` and `adminonly_admin1` are defined

# Convert the lists to sets for comparison
set_fewsnet = set(admin_name_only_fewsnet)
set_admin1 = set(admin_name_only_admin1)
set_admin2 = set(admin_name_only_admin2)

# Find the intersection (matches)
matches_toadmin1 = set_fewsnet.intersection(set_admin1)
matches_toadmin2 = set_fewsnet.intersection(set_admin2)

# Print the matches and their count
print("Matches between adminonly_fewsnet and adminonly_admin1:")
print(matches_toadmin1)
print(f"\nNumber of matches: {len(matches_toadmin1)}")

# Print the matches and their count
print("Matches between adminonly_fewsnet and adminonly_admin2:")
print(matches_toadmin2)
print(f"\nNumber of matches: {len(matches_toadmin2)}")


Matches between adminonly_fewsnet and adminonly_admin1:
set()

Number of matches: 0
Matches between adminonly_fewsnet and adminonly_admin2:
set()

Number of matches: 0


### Check the field types of each dataframe before perfoming a join

In [35]:
display(subset_fewsnet.head(3))
print('Field types:')
print(subset_fewsnet.dtypes)
print('Unique years:')
print(admin1_eff_years)
print()
print('Field types:')
display(subset_admin1.head(3))
print()
print(subset_admin1.dtypes)
print('Field types:')
display(subset_admin2.head(3))
print(subset_admin2.dtypes)
print('Unique years:')
print(admin2_eff_years)

,fnid,reporting_date,year
0,ET2021C1020203,2023-04-01,2023
2,ET2021C1020209,2023-04-01,2023
4,ET2021C1041904,2023-04-01,2023


Field types:
fnid              object
reporting_date    object
year              object
dtype: object
Unique years:
[0, 1962, 1980, 1990, 1994, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2017]

Field types:


,FNID,EFF_YEAR
0,AO2008A101,2008
1,AO2008A102,2008
2,AO2008A103,2008



FNID        object
EFF_YEAR     int64
dtype: object
Field types:


,FNID,EFF_YEAR
0,AO2008A21701,2008
1,AO2008A21501,2008
2,AO2008A20901,2008


FNID        object
EFF_YEAR     int64
dtype: object


#### Change field types to 'string' type:

In [24]:
# Convert 'EFF_YEAR' to string type in subset_admin1
subset_admin1['EFF_YEAR'] = subset_admin1['EFF_YEAR'].astype(str)

# Convert 'EFF_YEAR' to string type in subset_admin2
subset_admin2['EFF_YEAR'] = subset_admin2['EFF_YEAR'].astype(str)

# Verify the change
print("Updated field types for subset_admin1:")
print(subset_admin1.dtypes)

print("\nUpdated field types for subset_admin2:")
print(subset_admin2.dtypes)


Updated field types for subset_admin1:
FNID        object
EFF_YEAR    object
dtype: object

Updated field types for subset_admin2:
FNID        object
EFF_YEAR    object
dtype: object


### Now perform join

- this will identify the rows that have a match for BOTH admin ID and year

In [37]:
# Merge with subset_admin1
merged_with_admin1 = subset_fewsnet.merge(
    subset_admin1,
    # left_on=['fnid', 'year'],
    left_on='fnid',
    # right_on=['FNID', 'EFF_YEAR'],
    right_on='FNID',
    how='left'  # Use 'left' join to retain all rows from subset_fewsnet
)

# Add 'Matched_to_level' field with value '1' where match occurs
merged_with_admin1['Matched_to_level'] = merged_with_admin1.apply(
    lambda row: '1' if not pd.isna(row['FNID']) else None, axis=1
)

# # Merge with subset_admin2
# merged_with_admin2 = merged_with_admin1.merge(
#     subset_admin2,
#     left_on=['fnid', 'year'],
#     right_on=['FNID', 'EFF_YEAR'],
#     how='left'
# )

# # Update 'Matched_to_level' to '2' for matches in subset_admin2
# merged_with_admin2['Matched_to_level'] = merged_with_admin2.apply(
#     lambda row: '2' if not pd.isna(row['FNID_y']) else row['Matched_to_level'], axis=1
# )

# Clean up the resulting DataFrame (optional)
# final_result = merged_with_admin2[['fnid', 'year', 'Matched_to_level']].drop_duplicates()

# Display the final result
print(merged_with_admin1)


                  fnid reporting_date  year FNID  EFF_YEAR Matched_to_level
0       ET2021C1020203     2023-04-01  2023  NaN       NaN             None
1       ET2021C1020209     2023-04-01  2023  NaN       NaN             None
2       ET2021C1041904     2023-04-01  2023  NaN       NaN             None
3       ET2021C1130109     2023-04-01  2023  NaN       NaN             None
4       ET2021C1050699     2023-04-01  2023  NaN       NaN             None
...                ...            ...   ...  ...       ...              ...
164108        TG2024C0     2024-07-01  2024  NaN       NaN             None
164109        TG2024C0     2024-08-01  2024  NaN       NaN             None
164110        TG2024C0     2024-09-01  2024  NaN       NaN             None
164111        TG2024C0     2024-04-01  2024  NaN       NaN             None
164112        TG2024C0     2024-06-01  2024  NaN       NaN             None

[164113 rows x 6 columns]


### Check for completeness of matches:

In [38]:
# Replace NaN values in 'Matched_to_level' with 'none' (if applicable)
merged_with_admin1['Matched_to_level'] = merged_with_admin1['Matched_to_level'].fillna('None')

# Count occurrences of each value in 'Matched_to_level'
matched_to_level_counts = merged_with_admin1['Matched_to_level'].value_counts()

# Ensure all expected values ('none', '1', '2') are included
expected_values = ['None', '1', '2']
for value in expected_values:
    if value not in matched_to_level_counts:
        matched_to_level_counts[value] = 0

# Sort and print the result
matched_to_level_counts = matched_to_level_counts[expected_values]
print("Counts of each value in 'Matched_to_level':")
print(matched_to_level_counts)


Counts of each value in 'Matched_to_level':
None    164113
1            0
2            0
Name: Matched_to_level, dtype: int64


In [29]:
# Replace NaN values in 'Matched_to_level' with 'none' (if applicable)
final_result['Matched_to_level'] = final_result['Matched_to_level'].fillna('None')

# Count occurrences of each value in 'Matched_to_level'
matched_to_level_counts = final_result['Matched_to_level'].value_counts()

# Ensure all expected values ('none', '1', '2') are included
expected_values = ['None', '1', '2']
for value in expected_values:
    if value not in matched_to_level_counts:
        matched_to_level_counts[value] = 0

# Sort and print the result
matched_to_level_counts = matched_to_level_counts[expected_values]
print("Counts of each value in 'Matched_to_level':")
print(matched_to_level_counts)


Counts of each value in 'Matched_to_level':
None    18647
1           0
2           0
Name: Matched_to_level, dtype: int64
